In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
dataset = pd.read_csv('creditcard.csv')
X = StandardScaler().fit_transform(dataset.drop('Class', axis=1))
y = dataset['Class']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Build the Autoencoder model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, InputLayer

input_dim, encoding_dim = X_train.shape[1], 16
encoder = Sequential([
    InputLayer(input_shape=(input_dim,)),
    Dense(32, activation='relu'),
    Dense(encoding_dim, activation='relu')
])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


In [ ]:
decoder = Sequential([
    InputLayer(input_shape=(encoding_dim,)),
    Dense(32, activation='relu'),
    Dense(input_dim, activation='sigmoid')
])

In [ ]:
autoencoder = Sequential([encoder, decoder])

autoencoder.compile(optimizer='adam', loss='mean_squared_error')

autoencoder.fit(X_train, X_train, epochs=10, batch_size=32, shuffle=True, validation_data=(X_test, X_test))

autoencoder.summary()

Epoch 1/10
1587/1587 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - loss: nan - val_loss: nan
Epoch 2/10
1587/1587 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: nan - val_loss: nan
Epoch 3/10
1587/1587 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: nan - val_loss: nan
Epoch 4/10
1587/1587 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - loss: nan - val_loss: nan
Epoch 5/10
1587/1587 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - loss: nan - val_loss: nan
Epoch 6/10
1587/1587 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: nan - val_loss: nan
Epoch 7/10
1587/1587 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: nan - val_loss: nan
Epoch 8/10
1587/1587 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: nan - val_loss: nan
Epoch 9/10
1587/1587 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - loss: nan - val_loss: nan
Epoch 10/10
1587/1587 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - loss: nan - val_loss: nan


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ sequential (Sequential)              │ (None, 16)                  │           1,520 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ sequential_1 (Sequential)            │ (None, 30)                  │           1,534 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 9,164 (35.80 KB)

 Trainable params: 3,054 (11.93 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 6,110 (23.87 KB)

In [ ]:
predictions = autoencoder.predict(X_test)
mse = np.mean(np.power(X_test - predictions, 2), axis=1)
threshold = np.percentile(mse, 95)

# Classify anomalies
y_pred = np.where(mse > threshold, 1, 0)

397/397 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step


In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

print(f'Accuracy: {accuracy_score(y_test, y_pred)}')
print(f'Confusion Matrix:\n{confusion_matrix(y_test, y_pred)}')
print(f'Classification Report:\n{classification_report(y_test, y_pred)}')

Accuracy: 0.9973217802284364
Confusion Matrix:
[[12661     0]
 [   34     0]]
Classification Report:
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00     12661
         1.0       0.00      0.00      0.00        34

    accuracy                           1.00     12695
   macro avg       0.50      0.50      0.50     12695
weighted avg       0.99      1.00      1.00     12695



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
